In [22]:
import numpy as np
import sympy
from collections import defaultdict
import itertools

In [2]:
a = 19349663
l = 19349600
u = 19349700
pr = list(sympy.primerange(l, u))
print(len(pr))

6


In [3]:
list(pr)

[19349609, 19349633, 19349639, 19349651, 19349653, 19349669]

In [4]:
[p - a for p in pr]

[-54, -30, -24, -12, -10, 6]

In [5]:
l = 5
n = 5
p1,p2,p3 = 73856093, 19349669, 83492791

x1 = [33,4,11]
x2 = [31,1,14]
x3 = [10,44,19]

def spatial_hash(x):
    ix,iy,iz = np.floor(x[0]/l), np.floor(x[1]/l), np.floor(x[2]/l)
    return (int(ix*p1) ^ int(iy*p2) ^ int(iz*p3)) % n

In [6]:
spatial_hash(x1)

1

In [7]:
spatial_hash(x2)

1

In [8]:
spatial_hash(x3)

2

In [9]:
def spatial_hash_3D(cell_size, table_size):
    px = np.array([73856093, 19349669, 83492791])
    def _hash(coord3D):
        cf = np.floor(np.atleast_2d(coord3D) / cell_size).astype(int)
        i0 = cf[:,0] * px[0]
        i1 = cf[:,1] * px[1]
        i2 = cf[:,2] * px[2]
        return  (i0 ^ i1 ^ i2) % table_size
    return _hash

In [10]:
def spatial_hash_3Dorg(cell_size, table_size):
    px = np.array([73856093, 19349669, 83492791])
    def _hash(coord3D):
        cf = np.floor(np.atleast_2d(coord3D) / cell_size)
        i0 = (cf[:,0] * px[0]).astype(int)
        i1 = (cf[:,1] * px[1]).astype(int)
        i2 = (cf[:,2] * px[2]).astype(int)
        return  (i0 ^ i1 ^ i2) % table_size
    return _hash

In [11]:
sp = spatial_hash_3D(l, n)

In [12]:
sp(x3)

array([2], dtype=int32)

In [13]:
xx = np.array([x1, x2, x3])
xx

array([[33,  4, 11],
       [31,  1, 14],
       [10, 44, 19]])

In [14]:
sp(xx)

array([1, 1, 2], dtype=int32)

In [15]:
class HashMap:
    def __init__(self, cell_size, table_size):
        self.cell_size = cell_size
        self.table_size = table_size
        self.contents = defaultdict(list)
        px = np.array([73856093, 19349669, 83492791])
        
    def _hash(self, coord3Ds):
        cf = np.floor(np.atleast_2d(coord3D) / cell_size).astype(int)
        i0 = cf[:,0] * px[0]
        i1 = cf[:,1] * px[1]
        i2 = cf[:,2] * px[2]
        return  (i0 ^ i1 ^ i2) % self.table_size
    
    def _bbox3D_keys(self, bbox3D):
        keys = []
        # rearrange to standardize bounding box coords
        bbox3D = np.array(self._hash(bbox3D.min(axis=0)), self._hash(bbox3D.max(axis=0)))
        # quantize the min/max corner points
        bbox3D = np.floor(bbox3D / self.cell_size).astype(int)
        # iterate over the rectangular region
        for i in range(bbox3D[0][0], bbox3D[1][0]+1):
            for j in range(bbox3D[0][1], bbox3D[1][1]+1):
                for k in range(bbox3D[0][2], bbox3D[1][2]+1):
                    # append to each intersecting cell
                    i0 = i * self.px[0]
                    i1 = j * self.px[1]
                    i2 = k * self.px[2]
                    keys.append((i0 ^ i1 ^ i2) % self.table_size)
        return keys
    
    @classmethod
    def bulk_load(cls, cell_size, table_size, coord3Ds, objs):
        if len(coord3Ds) != len(objs):
            raise ValueError('coord3Ds and objs must be the same length')
        hm = HashMap(cell_size, table_size)
        hm.contents = defaultdict(list, zip(self._hash(coord3Ds), objs))
    
    def insert_object_for_point(self, coord3D, obj):
        key = self._hash([coord3Ds]) # get the hash keys for the cooridinates
        self.contents[self._hash([coord3Ds])[0]].append(obj)    
    
    def insert_object_for_aabb3D(self, bbox3D, obj):
        keys = self._bbox3D_keys(self, bbox3D)
        [self.contents[key].append(obj) for key in keys]
    
    def get_object_from_aabb3D(self, bbox3D):
        keys = self._bbox3D_keys(self, bbox3D)
        objs = set([objs.add(self.contents[key]) for key in keys])
        return objs

In [16]:
bb = np.array([[1, 20, 30], [10, 2, 3]])

In [17]:
bb

array([[ 1, 20, 30],
       [10,  2,  3]])

In [18]:
bb.min(axis=0)

array([1, 2, 3])

In [20]:
arrs = ([1, 2, 3], [4, 5], [6, 7])

In [25]:
list(itertools.product(*arrs))

[(1, 4, 6),
 (1, 4, 7),
 (1, 5, 6),
 (1, 5, 7),
 (2, 4, 6),
 (2, 4, 7),
 (2, 5, 6),
 (2, 5, 7),
 (3, 4, 6),
 (3, 4, 7),
 (3, 5, 6),
 (3, 5, 7)]

In [26]:
testdata ='xxx'